Linear programming (continuous) and binary linear (discrete) programming models for lower-bounding and computing the frustration index of relatively dense signed graphs

Jupyter code written by Samin Aref in 2018

 Creative common licence: 
 Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)

Using this code for non-commercial purposes is permitted to all given that the three following publications are cited:

1. Aref, S, Mason, AJ, Wilson, MC. A modeling and computational study of the frustration index in signed networks. Networks. 2020; 75: 95– 110. url: https://doi.org/10.1002/net.21907


2. Aref, S., Mason, A. J., and Wilson, M. C., Computing the line index of balance using integer programming optimisation. In Optimization Problems in Graph Theory, B. Goldengorin, Ed. Springer, 2018, pp. 65-84. url: https://doi.org/10.1007/978-3-319-94830-0_3


3. Aref, S., and Neal, Z., Detecting Coalitions by Optimally Partitioning Signed Networks of Political Collaboration. Scientific Reports (2020). url: http://arxiv.org/pdf/1906.01696.
        

Related dataset: 
    
    Neal, Z., A Sign of the Times: Dataset of US Congress signed network backbones from co-sponsorship data, 1973-2016. Figshare research data repository (2019). http://doi.org/10.6084/m9.figshare.8096429.v3. 
        
Suggested articles and other relevant datasets:

    Aref, S., and Wilson, M. C. Balance and frustration in signed networks. Journal of Complex Networks 7, 2 (2019), 163–189. doi: 10.1093/comnet/cny015.
    
    Aref, S., and Wilson, M. C. Measuring partial balance in signed networks. Journal of Complex Networks 6, 4 (2018), 566-595. doi: 10.1093/comnet/cnx044. 
    
    Aref, S. Signed networks from sociology and political science, systems biology, international relations, finance, and computational chemistry. Figshare research data repository (2017). doi: 10.6084/m9.figshare.5700832.v2.

In [14]:
# Constructing SEVERAL signed graphs from data file in excel format

# This is coded for the ease of users to load the data exactly from
# the file provided in the FigShare repository. However, loading networks
# from .csv format is much faster (code is provided in the end). 

import networkx as nx
import csv
import numpy as np
import pandas as pd

signedMatrix=[]
unsignedMatrix=[]
Graph=[]
sorted_weighted_edges=[]

chamber=input("House or Senate? (H/S)")

start_year=int(input("start session (inclusive)? (96,97,98,...,114)"))
end_year=int(input("end session (exclusive)? (97,98,99,...,115)"))

n=end_year-start_year

for index in range(n):
    
    line=pd.read_excel('datasets\\a_sign_of_the_times.xlsx',\
                       sheet_name=str(chamber.upper()+str(index+start_year)), header=None, nrows = 1)
    cols=range(1,len(line.columns))
    adjacency=pd.read_excel('datasets\\a_sign_of_the_times.xlsx',\
                            sheet_name=str(chamber.upper()+str(index+start_year)), header=None, usecols=cols)
    matrix= adjacency.rename_axis('ID').values
    order=len(np.matrix(matrix))
    
    signedMatrix.append(np.matrix(matrix))
    unsignedMatrix.append(abs(np.matrix(matrix)))
    
    Graph.append(nx.from_numpy_matrix(signedMatrix[index]))
    sorted_weighted_edges.append(nx.get_edge_attributes(Graph[index], 'weight'))
    
    print("instance",chamber.upper(),index+start_year,"generated")
    
run=len(Graph)    

House or Senate? (H/S)s
start session (inclusive)? (96,97,98,...,114)96
end session (exclusive)? (97,98,99,...,115)98
instance S 96 generated
instance S 97 generated


In [5]:
# Using the party affiliation of legislators to create a feasible solution (upper bound on frustration index)
# to "warm-start" the algorithm
# This step is optional, but if you skip this, you should comment out five lines of the code for continuous model

import pandas as pd

party=[]
dataframes=[]
control=[]
for index in range(n):
    cols=range(1)
    df=pd.read_excel('datasets\\a_sign_of_the_times.xlsx',\
                            sheet_name=str(chamber.upper()+str(index+start_year)), header=None, usecols=cols)
    dataframes.append(df)
    control.append([len(df[df[0].str.contains("-D")]),\
                    len(df[df[0].str.contains("-R")]),\
                   len(df)-len(df[df[0].str.contains("-D")])-len(df[df[0].str.contains("-R")])])

#print('Democrats')
#for index in range(n):
#    print((control[index])[0])
    
#print('Republicans')
#for index in range(n):
#    print((control[index])[1])
        
#print('Others')
#for index in range(n):
#    print((control[index])[2])
    
for index in range(n):
    party.append(list((dataframes[index])[0].str.contains("-D")))

Democrats
59
47
Republicans
41
53
Others
1
1


In [6]:
# Continuous model to obtain a lower bound for L(G) as the optimal objective function
# Based on the linear programming model (Eq. 7) discussed in the following publication
#  Aref, S., and Neal, Z., Detecting Coalitions by Optimally Partitioning Signed Networks 
#  of Political Collaboration. Scientific Reports (2020). url: http://arxiv.org/pdf/1906.01696.


# This code solves graph optimization model(s) using "Gurobi solver"
# to compute a lower-bound of frustration index for the input signed graph(s)

# Note that you must have installed Gurobi into Jupyter and registered a Gurobi license
# in order to run this code


import time
import numpy as np
import multiprocessing
from gurobipy import *
objectivevalue=[]
solveTime=[]

for index in range(run):
    
    order=len(signedMatrix[index])
    NumberOfNegative=((-1 == signedMatrix[index])).sum()/2
    size=int(np.count_nonzero(signedMatrix[index])/2)

    neighbors={}
    Degree=[]
    for u in sorted((Graph[index]).nodes()):
        neighbors[u] = list((Graph[index])[u])
        Degree.append(len(neighbors[u]))
    unsignedDegree=Degree
    
    #Finding the node with the highest unsigned degree
    maximum_degree = max(unsignedDegree)
    [node_to_fix]=[([i for i, j in enumerate(unsignedDegree) if j == maximum_degree]).pop()]

    # Model parameters
    model = Model("Continuous model for lower-bounding frustration index")
    
    # There are different methods for solving optimization models:
    # (-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent)
    # For problems with a large number of contstraints, barrier method is more suitable
    model.setParam(GRB.param.Method, 2)

    # Solving the problems without crossover takes a substantially shorter time
    # in cases where there are a large number of constraints. (0=without, 1=with)
    model.setParam(GRB.Param.Crossover, 0)
    
    # What is the time limit in second?
    # Here, it is set to 10 hours
    model.setParam('TimeLimit', 10*3600)
    
    # Do you want details of branching to be reported? (0=No, 1=Yes)
    model.setParam(GRB.param.OutputFlag, 1) 
    
    # How many threads to be used for exploring the feasible space in parallel?
    # Here, the minimum of 32 and the availbale CPUs is used
    model.setParam(GRB.Param.Threads, min(32,multiprocessing.cpu_count()))
   
    #This chunk of code lists the graph triangles
    GraphTriangles=[]
    for n1 in sorted((Graph[index]).nodes()):
        neighbors1 = set((Graph[index])[n1])
        for n2 in filter(lambda x: x>n1, neighbors1):
            neighbors2 = set((Graph[index])[n2])
            common = neighbors1 & neighbors2
            for n3 in filter(lambda x: x>n2, common):
                GraphTriangles.append([n1,n2,n3])
    len(GraphTriangles)
    #print("--- %Listed",len(GraphTriangles),"triangles for the graph")
    

    # Create decision variables and update model to integrate new variables
    # Note that the variables are defined as CONTINUOUS within the unit interval
    x=[]
    for i in range(0,order):
        x.append(model.addVar(lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='x'+str(i)))

    z={}    
    for (i,j) in (sorted_weighted_edges[index]):
        z[(i,j)]=model.addVar(lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='z'+str(i)+','+str(j))    

    model.update()
    
    # Set the objective function
    OFV=0
    for (i,j) in (sorted_weighted_edges[index]):
        OFV = OFV + (1-(sorted_weighted_edges[index])[(i,j)])/2 + ((sorted_weighted_edges[index])[(i,j)])*(x[i]+x[j]-2*z[(i,j)])          
    model.setObjective(OFV, GRB.MINIMIZE)

    # Add constraints to the model and update model to integrate new constraints
    
    ## ADD CORE CONSTRAINTS ##

    for (i,j) in (sorted_weighted_edges[index]):
            if (sorted_weighted_edges[index])[(i,j)]==1:
                model.addConstr(z[(i,j)] <= (x[i]+x[j])/2 , 'Edge positive'+str(i)+','+str(j))
            if (sorted_weighted_edges[index])[(i,j)]==-1:
                model.addConstr(z[(i,j)] >= x[i] + x[j] -1 , 'Edge negative'+str(i)+','+str(j))            

    for triangle in GraphTriangles:
            [i,j,k]=triangle
            model.addConstr(x[j] + z[(i,k)] >= z[(i,j)] + z[(j,k)] , 'triangle1'+','+str(i)+','+str(j)+','+str(k))
            model.addConstr(x[i] + z[(j,k)] >= z[(i,j)] + z[(i,k)] , 'triangle2'+','+str(i)+','+str(j)+','+str(k))       
            model.addConstr(x[k] + z[(i,j)] >= z[(i,k)] + z[(j,k)] , 'triangle3'+','+str(i)+','+str(j)+','+str(k))
            model.addConstr( 1 + z[(i,j)] + z[(i,k)] + z[(j,k)] >= x[i] + x[j] + x[k] , 'triangle4'+','+str(i)+','+str(j)+','+str(k))           
    model.update()

    ## ADD ADDITIONAL CONSTRAINTS (speed-ups) ##
    
    # Colour the node with the highest degree as 1
    model.addConstr(x[node_to_fix]==1 , '1stnodecolour')   
    model.update()
  

    # Solve
    start_time = time.time()
    model.optimize()
    solveTime.append(time.time() - start_time) 
    
    # Save optimal objective function values
    obj = model.getObjective()
    objectivevalue.append((obj.getValue()))
        
    # Report the optimal objective function value for each instance
    print('Instance', index,' solution equals',np.around(objectivevalue[index])) 
    print("-"*92)
    
    # Printing the solution (optional)
    #print("Optimal values of the decision variables")
    #for v in model.getVars():
    #    print (v.varName, v.x)
    #print()    

# Save the lower bounds as a list for the next step (computing the frutsration index)
LowerBounds=np.around(objectivevalue)  

print("-"*32,"***  EXPERIMENT STATS  ***","-"*32)
print("-"*92)
print("Lower bounds on frustration index:",LowerBounds)
print()
print("Solve times:",np.around(solveTime, decimals=2))
print("Average solve time",np.mean(solveTime))
#print("Solve time Standard Deviation",np.std(solveTime))
print()

Academic license - for non-commercial use only
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Crossover to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter TimeLimit to 36000.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 88036 rows, 2376 columns and 392746 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.18s
Presolved: 2375 rows, 90410 columns, 390751 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.331e+05
 Factor NZ  : 1.544e+06 (roughly 50 MBytes of memory)
 Factor Ops : 1.461e+09 (less than 1 second per iteration)
 Threads    :

In [7]:
# Discrete model to be given lower bounds and compute L(G) as the optimal objective function
# Based on the binary linear programming model (Eq. 8) discussed in the following publication
#  Aref, S., and Neal, Z., Detecting Coalitions by Optimally Partitioning Signed Networks 
#  of Political Collaboration. Scientific Reports (2020). url: http://arxiv.org/pdf/1906.01696.

# This code solves graph optimization model(s) using "Gurobi solver"
# to compute the measure called, frustration index, for the input signed graph(s)

# Note that you must have installed Gurobi into Jupyter and registered a Gurobi license
# in order to run this code


#Setting parameters
#lazyParam=int(input("What is the lazy parameter for unbalanced triangle lazy cuts? (0/1/2/3)"))
# See "lazy" as a tunable parameter in linear constraint attributes in Gurobi optimizer reference manual below:
# https://www.gurobi.com/documentation/8.1/refman/lazy.html
lazyParam=int(2)

#speedupParam=int(input("Do you want to use the speedups? (0=No, 1=Yes)"))
speedupParam=int(1)

import time
import numpy as np
import multiprocessing
from gurobipy import *
objectivevalue=[]
solveTime=[]
effectiveBranchingFactors=[]

for index in range(run):
    
    order=len(signedMatrix[index])
    NumberOfNegative=((-1 == signedMatrix[index])).sum()/2
    size=int(np.count_nonzero(signedMatrix[index])/2)

    neighbors={}
    Degree=[]
    for u in sorted((Graph[index]).nodes()):
        neighbors[u] = list((Graph[index])[u])
        Degree.append(len(neighbors[u]))
    unsignedDegree=Degree
    
    #Finding the node with the highest unsigned degree
    maximum_degree = max(unsignedDegree)
    [node_to_fix]=[([i for i, j in enumerate(unsignedDegree) if j == maximum_degree]).pop()]

    # Model parameters
    model = Model("Continuous model for computing the frustration index (a lower bound is needed)")

    # There are different methods for solving optimization models:
    # (-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent)
    # model.setParam(GRB.param.Method, -1)
    
    # What is the time limit in second?
    # model.setParam('TimeLimit', 10*3600)
    
    # Do you want details of branching to be reported? (0=No, 1=Yes)
    model.setParam(GRB.param.OutputFlag, 1) 
    
    # Do you want a non-zero Mixed integer programming tolerance (MIP Gap)?
    # Note that a non-zero MIP gap may prevent the model from computing the exact value of frustration index
    # model.setParam('MIPGap', 0.0001)  
    
    # How many threads to be used for exploring the feasible space in parallel?
    # Here, the minimum of 32 and the availbale CPUs is used
    model.setParam(GRB.Param.Threads, min(32,multiprocessing.cpu_count()))
    
    #This chunk of code lists the graph triangles
    GraphTriangles=[]
    for n1 in sorted((Graph[index]).nodes()):
        neighbors1 = set((Graph[index])[n1])
        for n2 in filter(lambda x: x>n1, neighbors1):
            neighbors2 = set((Graph[index])[n2])
            common = neighbors1 & neighbors2
            for n3 in filter(lambda x: x>n2, common):
                GraphTriangles.append([n1,n2,n3])
    len(GraphTriangles)
    #print("--- %Listed",len(GraphTriangles),"triangles for the graph")

    #This chunk of code lists the balanced and unbalanced triangles
    w=nx.get_edge_attributes(Graph[index], 'weight')  
    unbalanced_triangles = []
    balanced_triangles = []
    for triad in GraphTriangles: 
        if  (sorted_weighted_edges[index])[(triad[0],triad[1])]*\
        (sorted_weighted_edges[index])[(triad[0],triad[2])]*(sorted_weighted_edges[index])[(triad[1],triad[2])] == -1:
            unbalanced_triangles.append(triad)
        elif (sorted_weighted_edges[index])[(triad[0],triad[1])]*\
        (sorted_weighted_edges[index])[(triad[0],triad[2])]*(sorted_weighted_edges[index])[(triad[1],triad[2])] == 1:
            balanced_triangles.append(triad)  
    

    # Create decision variables and update model to integrate new variables
    x=[]
    for i in range(0,order):
        x.append(model.addVar(vtype=GRB.BINARY, name='x'+str(i))) # arguments by name
    model.update()
    
    f={}
    for (i,j) in (sorted_weighted_edges[index]):
        f[(i,j)]=model.addVar(lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='f'+str(i)+','+str(j))
    model.update()

    # Set the objective function
    OFV=0
    for (i,j) in (sorted_weighted_edges[index]):
        OFV = OFV + f[(i,j)]                 
    model.setObjective(OFV, GRB.MINIMIZE)

    # Add constraints to the model and update model to integrate new constraints
    
    ## ADD CORE CONSTRAINTS ##

    for (i,j) in (sorted_weighted_edges[index]):
        model.addConstr( f[(i,j)] >= x[i] - ((sorted_weighted_edges[index])[(i,j)])*x[j] -\
                        (1-(sorted_weighted_edges[index])[(i,j)])/2
                             , '1st Edge'+','+str(i)+','+str(j))
        model.addConstr( f[(i,j)] >= -x[i] + ((sorted_weighted_edges[index])[(i,j)])*x[j] +\
                        (1-(sorted_weighted_edges[index])[(i,j)])/2
                             , '2nd Edge'+','+str(i)+','+str(j))                  
    model.update()  
    
    model.addConstr(OFV >= int(LowerBounds[index]), 'LP lower bound')
    model.update()
            
    ## ADD ADDITIONAL CONSTRAINTS (speed-ups) ##
    
    if speedupParam==1:

        # Triangle valid inequalities            

        triangleInequalityCount=len(unbalanced_triangles)
        for triangle in unbalanced_triangles:
            [i,j,k]=triangle
            model.addConstr(f[(i,j)] + f[(i,k)] + f[(j,k)] >= 1 ,'UnbalancedTriangle'+','+str(i)+','+str(j)+','+str(k))    

        model.update()
        model.setAttr('Lazy',model.getConstrs()[2*size:2*size+triangleInequalityCount],[lazyParam]*triangleInequalityCount)
        model.update() 

        #branching priority is based on unsigned degree 
        model.setAttr('BranchPriority',model.getVars()[:order],unsignedDegree)
        model.update()
        
    # Using a warm start
    # If a specific partitioning is known which can be used as a starting point to "warm-start" the algorithm,
    # it can be provided to the model here by giving start values to the node variables:
    # If the optional step above is skipped, you should comment out these five lines
    for i in range(0,order):
        if (party[index])[i]==True:
            x[i].start = 1.0
        else:
            x[i].start = 0    
    

    # Solve
    start_time = time.time()
    model.optimize()
    solveTime.append(time.time() - start_time) 
    
    
    # Save optimal objective function values
    obj = model.getObjective()
    objectivevalue.append((obj.getValue()))
    
    # Compute the effective branching factors
    if (model.NodeCount)**(1/(size+2*order)) >= 1:
        effectiveBranchingFactors.append((model.NodeCount)**(1/(size+2*order)))
        
    # Report the optimal objective function value for each instance
    print('Instance', index,' solution equals',objectivevalue[index]) 
    print("-"*92)
    
    # Printing the solution (optional)
    #print("Optimal values of the decision variables")
    #for v in model.getVars():
    #    print (v.varName, v.x)
    #print()    

  
    
print("-"*32,"***  EXPERIMENT STATS  ***","-"*32)
print("-"*92)
print("Frustration indices:",np.around(objectivevalue))
#print("Average frustrarion index",np.mean(objectivevalue))
#print("Frustration index Standard Deviation",np.std(objectivevalue))
print()
print("Solve times:",np.around(solveTime, decimals=2))
print("Average solve time",np.mean(solveTime))
#print("Solve time Standard Deviation",np.std(solveTime))
print()
if (model.NodeCount)**(1/(size+2*order)) >= 1:
    print("Effective branching factors (EBFs)",effectiveBranchingFactors)
    print("Average EBF",np.mean(effectiveBranchingFactors))
    #print("EBF Standard Deviation",np.std(effectiveBranchingFactors))

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 16982 rows, 2376 columns and 53218 nonzeros
Variable types: 2275 continuous, 101 integer (101 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]

Loaded MIP start with objective 882

Presolve time: 0.07s
Presolved: 16982 rows, 2376 columns, 53218 nonzeros
Extracted 12431 lazy constraints
Variable types: 2275 continuous, 101 integer (101 binary)

Root relaxation: objective 1.000000e+00, 106 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0  101  882.00000    1.00000   100%     -    0s
H    0     0                     658.0000000    1.00000   100% 

Two extra chunks of code are provided below for:

 (1) generating synthetic data using random graph models,
 
 (2) loading networks of US Congress legislators from .csv files.

In [8]:
# This code is for producing synthetic data (generating random "signed graphs")
# "Signed graphs" are graphs in which edges are postive or negative (signed)
# This code asks you for type of random graph, several parameters for generating them such as size and order
# and then it asks for "number of negative edges" so that it randomly makes some edges negative and produce 
# a signed graph out of an unsigned graph

import networkx as nx
import numpy as np
import random


listOfGraphs=[]
NumberOfNegative=[]
run=int(input("How many random graphs do you want to create?(1/2/...)"))

print("\n 1.  Erdős-Rényi binomial graph G_{n,p} "
      "\n 2.  Random graph of n nodes and m total edges G_{n,m} "
      "\n 3.  Newman-Watts-Strogatz small world graph (adding new edges to the ring) G_{n,k,p} "
      "\n 4.  Watts-Strogatz small world graph (rewiring the ring) G_{n,k,p} "
      "\n 5.  Random regular graph of n nodes each with degree d G_{d,n} "
      "\n 6.  Barabási-Albert preferential attachment model graph G_{n,m} "
      "\n 7.  Growing graph with powerlaw degree distribution and approximate average clustering G_{n,m,p}"
      "\n 8.  Two-dimensional grid n*n ")

graph_type=int(input("What type of random graph(s) do you want to create from the list above? (1/2/...)"))

for i in range(run):

    if graph_type == 1:
        n=int(input("What is the number of nodes?"))
        p=float(input("What is the probability of having an edge for each two nodes? (0.1, 0.5, ...)"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.fast_gnp_random_graph(n,p)
        listOfGraphs.append(G) 
    elif graph_type == 2:
        n=int(input("What is the number of nodes?"))
        m=int(input("What is the total number of edges?"))  
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.gnm_random_graph(n,m)
        listOfGraphs.append(G)
    elif graph_type == 3:
        n=int(input("What is the number of nodes?"))
        k=int(input("What is the number of nearest neighbors in ring topology?"))
        p=float(input("What is the probability of adding a new edge?"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.newman_watts_strogatz_graph(n,k,p)
        listOfGraphs.append(G)
    elif graph_type == 4:
        n=int(input("What is the number of nodes?"))
        k=int(input("What is the number of nearest neighbors in ring topology?"))
        p=float(input("What is the probability of rewiring each edge?"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.watts_strogatz_graph(n,k,p)
        listOfGraphs.append(G)
    elif graph_type == 5:
        n=int(input("What is the number of nodes?(n>d)"))
        d=int(input("What is the degree?(The value of n*d must be even)"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.random_regular_graph(d,n)
        listOfGraphs.append(G)
    elif graph_type == 6:
        n=int(input("What is the number of nodes?"))
        m=int(input("What is the number of edges to attach from a new node to existing nodes?"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.barabasi_albert_graph(n,m)   
        listOfGraphs.append(G)
    elif graph_type == 7:
        n=int(input("What is the number of nodes?"))
        m=int(input("What is the number of random edges to add for each new node?"))
        p=float(input("What is the probability of adding a triangle after adding a random edge?"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.powerlaw_cluster_graph(n,m,p)    
        listOfGraphs.append(G)
    elif graph_type == 8:
        n=int(input("What is the size of the grid?(sqrt(n))"))
        NumberOfNegative.append(int(input("What is the number of negative edges for this instance? (0/1/2/.../m)")))
        G=nx.grid_2d_graph(n,n)
        mapping=dict(zip(G.nodes(),range(len(G.nodes()))))
        G=nx.relabel_nodes(G,mapping)
        listOfGraphs.append(G)
    else:
        print("Try again and choose a graph type from the selection above!")

                    
Graph=[]
signedMatrix=[]
unsignedMatrix=[]
ShuffledEdges=[]
sorted_weighted_edges=[]


for i in range(run):
        ShuffledEdges = list(listOfGraphs[i].edges())
        random.shuffle(ShuffledEdges)
        Graph.append(nx.Graph())
        for u in listOfGraphs[i].nodes():
            Graph[i].add_node(u)
        for ind in range(0,NumberOfNegative[i]):
            (u,v) = ShuffledEdges[ind]
            Graph[i].add_edge(u,v,weight=-1)
        for ind in range(NumberOfNegative[i],len(listOfGraphs[i].edges())):
            (u,v) = ShuffledEdges[ind]
            Graph[i].add_edge(u,v,weight=1)
        signedMatrix.append(nx.to_numpy_matrix(Graph[i]))
        unsignedMatrix.append(abs(signedMatrix[i]))
        ShuffledEdges=[]

        weighted_edges=nx.get_edge_attributes(Graph[i], 'weight') 
        sorted_weighted_edges.append({})
        for (u,v) in weighted_edges:
            if u<v:
                (sorted_weighted_edges[i])[(u,v)] = weighted_edges[(u,v)]
            if u>v:
                (sorted_weighted_edges[i])[(v,u)] = weighted_edges[(u,v)]          
          

How many random graphs do you want to create?(1/2/...)2

 1.  Erdős-Rényi binomial graph G_{n,p} 
 2.  Random graph of n nodes and m total edges G_{n,m} 
 3.  Newman-Watts-Strogatz small world graph (adding new edges to the ring) G_{n,k,p} 
 4.  Watts-Strogatz small world graph (rewiring the ring) G_{n,k,p} 
 5.  Random regular graph of n nodes each with degree d G_{d,n} 
 6.  Barabási-Albert preferential attachment model graph G_{n,m} 
 7.  Growing graph with powerlaw degree distribution and approximate average clustering G_{n,m,p}
 8.  Two-dimensional grid n*n 
What type of random graph(s) do you want to create from the list above? (1/2/...)1
What is the number of nodes?50
What is the probability of having an edge for each two nodes? (0.1, 0.5, ...)0.5
What is the number of negative edges for this instance? (0/1/2/.../m)10
What is the number of nodes?50
What is the probability of having an edge for each two nodes? (0.1, 0.5, ...)0.5
What is the number of negative edges for this insta

In [13]:
# Constructing SEVERAL signed graphs from matrices in CSV format
# Loading networks from .csv format is much faster

import networkx as nx
import csv
import numpy as np

signedMatrix=[]
unsignedMatrix=[]
Graph=[]
sorted_weighted_edges=[]

chamber=input("House or Senate? (H/S)")

start_year=int(input("start session (inclusive)? (96,97,98,...,114)"))
end_year=int(input("end session (exclusive)? (97,98,99,...,115)"))

start_time = time.time()

n=end_year-start_year

for index in range(n):
    
    matrix=np.loadtxt(open('csv_datasets\\'+chamber.upper()+str(index+start_year)+'.csv', "rb"), delimiter=",")
    order=len(np.matrix(matrix))
    
    signedMatrix.append(np.matrix(matrix))
    unsignedMatrix.append(abs(np.matrix(matrix)))
    
    Graph.append(nx.from_numpy_matrix(signedMatrix[index]))
    sorted_weighted_edges.append(nx.get_edge_attributes(Graph[index], 'weight'))
    
    print("instance",chamber.upper(),index+start_year,"generated")

print("Total elapsed time:",(time.time() - start_time),"seconds")
    
run=len(Graph)    

House or Senate? (H/S)h
start session (inclusive)? (96,97,98,...,114)96
end session (exclusive)? (97,98,99,...,115)115
instance H 96 generated
instance H 97 generated
instance H 98 generated
instance H 99 generated
instance H 100 generated
instance H 101 generated
instance H 102 generated
instance H 103 generated
instance H 104 generated
instance H 105 generated
instance H 106 generated
instance H 107 generated
instance H 108 generated
instance H 109 generated
instance H 110 generated
instance H 111 generated
instance H 112 generated
instance H 113 generated
instance H 114 generated
Total elapsed time: 12.554391145706177 seconds
